### Prepare Deploy
* save the model to the model catalog and prepare deployment as **REST service**

In [18]:
import os
from ads.common.model_artifact import ModelArtifact
from ads.common.model_export_util import prepare_generic_model
from ads.common.model_metadata import (MetadataCustomCategory,
                                       UseCaseType)
from ads import set_auth

import json
import pickle

In [2]:
 # env and auth
compartment_id = os.environ['NB_SESSION_COMPARTMENT_OCID']
project_id = os.environ['PROJECT_OCID']

set_auth(auth='resource_principal')

In [3]:
# 1. prepare artifacts directory

PATH_ARTEFACT = f"./model-files"

if not os.path.exists(PATH_ARTEFACT):
    os.mkdir(PATH_ARTEFACT)

In [8]:
MODEL_FILE_NAME = "credit-scoring.pkl"
SCALER_FILE_NAME = "scaler.pkl"

model = pickle.load(open(PATH_ARTEFACT + "/" + MODEL_FILE_NAME, 'rb'))
scaler = pickle.load(open(PATH_ARTEFACT + "/" + SCALER_FILE_NAME, 'rb'))

In [24]:
# 2. prepare deploy to Model Catalog
# this URL is taken from the Conda published env
INFERENCE_ENV = "oci://conda_ds@frqap2zhtzbe/conda_environments/cpu/General Machine Learning for CPUs on Python 3.7/1.0/generalml_p37_cpu_v1"

artifact = prepare_generic_model(model=model, model_path=PATH_ARTEFACT,
                                 inference_conda_env=INFERENCE_ENV,
                                 force_overwrite=True, data_science_env=False,
                                 use_case_type=UseCaseType.BINARY_CLASSIFICATION)

loop1:   0%|          | 0/4 [00:00<?, ?it/s]

### Customize score.py

any customization you want to add to the main code f the REST service should be generated in this cell below

before executing artifact.save()

In [25]:
%%writefile {PATH_ARTEFACT}/score.py

from xgboost import XGBClassifier

import json
import os
from cloudpickle import cloudpickle
from functools import lru_cache

import io
import logging 

# logging configuration - OPTIONAL 
logging.basicConfig(format='%(name)s - %(levelname)s - %(message)s', level=logging.INFO)
logger_pred = logging.getLogger('model-prediction')
logger_pred.setLevel(logging.INFO)
logger_feat = logging.getLogger('input-features')
logger_feat.setLevel(logging.INFO)

model_name = 'credit-scoring.pkl'


"""
   Inference script. This script is used for prediction by scoring server when schema is known.
"""


@lru_cache(maxsize=10)
def load_model(model_file_name=model_name):
    """
    Loads model from the serialized format

    Returns
    -------
    model:  a model instance on which predict API can be invoked
    """
    model_dir = os.path.dirname(os.path.realpath(__file__))
    contents = os.listdir(model_dir)
    
    # Load the model from the model_dir using the appropriate loader
    
    if model_file_name in contents:
        with open(os.path.join(os.path.dirname(os.path.realpath(__file__)), model_file_name), "rb") as file:
            model = pickle.load(file) 
            logger_pred.info("Loaded model...")
       
    else:
        raise Exception('{0} is not found in model directory {1}'.format(model_file_name, model_dir))


def pre_inference(data):
    """
    Preprocess data

    Parameters
    ----------
    data: Data format as expected by the predict API of the core estimator.

    Returns
    -------
    data: Data format after any processing.

    """
    logger_pred.info("Eventually preprocessing and adding features...")
    
    return data

def post_inference(yhat):
    """
    Post-process the model results

    Parameters
    ----------
    yhat: Data format after calling model.predict.

    Returns
    -------
    yhat: Data format after any processing.

    """
    logger_pred.info("Eventually preprocessing output...")
    
    return yhat

def predict(data, model=load_model()):
    """
    Returns prediction given the model and data to predict

    Parameters
    ----------
    model: Model instance returned by load_model API
    data: Data format as expected by the predict API of the core estimator. For eg. in case of sckit models it could be numpy array/List of list/Pandas DataFrame

    Returns
    -------
    predictions: Output from scoring server
        Format: {'prediction': output from model.predict method}

    """
    features = pre_inference(data)
    yhat = post_inference(
        model.predict(features)
    )
    return {'prediction': yhat}

Overwriting ./model-files/score.py


### Save the model to the Model Catalog

In [21]:
# Saving the model artifact to the model catalog.
catalog_entry = artifact.save(display_name='credit-scoring', 
              description='A model for credit scoring')

loop1:   0%|          | 0/5 [00:00<?, ?it/s]

artifact:/tmp/saved_model_9ee9fac8-7bc4-486f-ae47-3a3afff7b5c8.zip
